Copyright 2025 Tsung-Tang Lee (also known as Osborn Lee or Moe)

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

http://www.apache.org/licenses/LICENSE-2.0

特別聲明：
禁止修改本程式碼後，聲稱為自己原創或作者。
修改後的作品必須保留原作者資訊

In [ ]:
import os
from pathlib import Path
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam

In [ ]:
print("TensorFlow version:", tf.__version__)
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))

TensorFlow version: 2.18.0
Num GPUs Available: 1


In [ ]:
!nvidia-smi

Sun Jul 20 09:17:43 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   47C    P8             12W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
batch_size = 128
num_classes = 10
epochs = 100
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

# The data, shuffled and split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# one-hot encoding
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

# build CNN model
model = Sequential([
    Conv2D(64, (3, 3), padding='same',input_shape=x_train.shape[1:]),
    Activation('relu'),
    Conv2D(128, (3, 3)),
    Activation('relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),

    Conv2D(64, (3, 3), padding='same'),
    Activation('relu'),
    Conv2D(64, (3, 3)),
    Activation('relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),

    Flatten(),
    Dense(512),
    Activation('relu'),
    Dropout(0.5),
    Dense(num_classes),
    Activation('softmax')
])

print(model.summary())

# initiate Adam optimizer
opt = Adam()

# train model by optimizer Adam
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255.
x_test /= 255.


print('data augmentation')
# Tpreprocessing : data augmentation
datagen = ImageDataGenerator(
    rotation_range=30,    # 訓練前，會被隨機旋轉 ±30 度之間的某個角度
    width_shift_range=0.1,  # 每張圖片在訓練前，會被隨機水平平移 ±10% 圖片寬度的距離
    height_shift_range=0.1,  # 每張圖片在訓練前，會被隨機上下平移 ±10% 圖片高度的距離
    horizontal_flip=True,   # 訓練時，每張圖片有機會被隨機水平翻轉（左右翻轉）
    vertical_flip=False)   # 在資料增強時，不執行垂直翻轉（上下顛倒）的操作

# Use ModelCheckpoint to save the best performance model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
checkpoint = ModelCheckpoint(model_path, monitor='val_loss', save_best_only=True, verbose=1)

# no earlystop
# Model fit
model_history = model.fit(datagen.flow(x_train, y_train,
                    batch_size=batch_size),
                    epochs=epochs,
                    validation_data=(x_test, y_test),
                    callbacks=[checkpoint],
                    )

# loading saved model that trained below
print("Loading trained model")
model = load_model(model_path)

# Score trained model by accuracy and loss
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step
x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 32, 32, 64)     │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 32, 32, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 30, 30, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_1 (Activation)       │ (None, 30, 30, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 15, 15, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 15, 15, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 15, 15, 64)     │        73,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_2 (Activation)       │ (None, 15, 15, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 13, 13, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_3 (Activation)       │ (None, 13, 13, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 6, 6, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 6, 6, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 2304)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_4 (Activation)       │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         5,130 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_5 (Activation)       │ (None, 10)             │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,371,658 (5.23 MB)

...

391/391 ━━━━━━━━━━━━━━━━━━━━ 31s 79ms/step - accuracy: 0.7846 - loss: 0.6271 - val_accuracy: 0.8177 - val_loss: 0.5415
Epoch 64/100
391/391 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - accuracy: 0.7818 - loss: 0.6243
Epoch 64: val_loss did not improve from 0.54151
391/391 ━━━━━━━━━━━━━━━━━━━━ 41s 79ms/step - accuracy: 0.7818 - loss: 0.6243 - val_accuracy: 0.8107 - val_loss: 0.5719
Epoch 65/100
391/391 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - accuracy: 0.7852 - loss: 0.6200
Epoch 65: val_loss did not improve from 0.54151
391/391 ━━━━━━━━━━━━━━━━━━━━ 30s 78ms/step - accuracy: 0.7852 - loss: 0.6200 - val_accuracy: 0.8157 - val_loss: 0.5431
Epoch 66/100
391/391 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - accuracy: 0.7819 - loss: 0.6322
Epoch 66: val_loss did not improve from 0.54151
391/391 ━━━━━━━━━━━━━━━━━━━━ 31s 79ms/step - accuracy: 0.7819 - loss: 0.6322 - val_accuracy: 0.7982 - val_loss: 0.6234
Epoch 67/100
391/391 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - accuracy: 0.7792 - loss: 0.6345
Epoch 67: val_loss did n

391/391 ━━━━━━━━━━━━━━━━━━━━ 31s 80ms/step - accuracy: 0.7895 - loss: 0.6067 - val_accuracy: 0.8296 - val_loss: 0.5230
Epoch 77/100
391/391 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step - accuracy: 0.7883 - loss: 0.6121
Epoch 77: val_loss did not improve from 0.52299
391/391 ━━━━━━━━━━━━━━━━━━━━ 32s 82ms/step - accuracy: 0.7883 - loss: 0.6122 - val_accuracy: 0.8011 - val_loss: 0.6189
Epoch 78/100
391/391 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - accuracy: 0.7854 - loss: 0.6208
Epoch 78: val_loss improved from 0.52299 to 0.51429, saving model to /content/saved_models/keras_cifar10_trained_model.h5


391/391 ━━━━━━━━━━━━━━━━━━━━ 31s 80ms/step - accuracy: 0.7854 - loss: 0.6208 - val_accuracy: 0.8286 - val_loss: 0.5143
Epoch 79/100
391/391 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step - accuracy: 0.7912 - loss: 0.6050
Epoch 79: val_loss did not improve from 0.51429
391/391 ━━━━━━━━━━━━━━━━━━━━ 32s 82ms/step - accuracy: 0.7912 - loss: 0.6050 - val_accuracy: 0.7965 - val_loss: 0.6533
Epoch 80/100
391/391 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - accuracy: 0.7919 - loss: 0.6118
Epoch 80: val_loss did not improve from 0.51429
391/391 ━━━━━━━━━━━━━━━━━━━━ 31s 79ms/step - accuracy: 0.7919 - loss: 0.6118 - val_accuracy: 0.8204 - val_loss: 0.5460
Epoch 81/100
391/391 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step - accuracy: 0.7938 - loss: 0.5945
Epoch 81: val_loss did not improve from 0.51429
391/391 ━━━━━━━━━━━━━━━━━━━━ 32s 81ms/step - accuracy: 0.7938 - loss: 0.5946 - val_accuracy: 0.8192 - val_loss: 0.5443
Epoch 82/100
391/391 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - accuracy: 0.7909 - loss: 0.6025
Epoch 82: val_loss did n

391/391 ━━━━━━━━━━━━━━━━━━━━ 31s 80ms/step - accuracy: 0.7962 - loss: 0.5964 - val_accuracy: 0.8318 - val_loss: 0.4976
Epoch 94/100
391/391 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step - accuracy: 0.7956 - loss: 0.5890
Epoch 94: val_loss did not improve from 0.49764
391/391 ━━━━━━━━━━━━━━━━━━━━ 31s 79ms/step - accuracy: 0.7956 - loss: 0.5890 - val_accuracy: 0.8219 - val_loss: 0.5431
Epoch 95/100
391/391 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - accuracy: 0.7995 - loss: 0.5821
Epoch 95: val_loss did not improve from 0.49764
391/391 ━━━━━━━━━━━━━━━━━━━━ 31s 80ms/step - accuracy: 0.7995 - loss: 0.5821 - val_accuracy: 0.8211 - val_loss: 0.5379
Epoch 96/100
391/391 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - accuracy: 0.7950 - loss: 0.5902
Epoch 96: val_loss did not improve from 0.49764
391/391 ━━━━━━━━━━━━━━━━━━━━ 31s 78ms/step - accuracy: 0.7950 - loss: 0.5902 - val_accuracy: 0.8144 - val_loss: 0.5547
Epoch 97/100
391/391 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - accuracy: 0.7929 - loss: 0.5911
Epoch 97: val_loss did n

Loading trained model
313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.8319 - loss: 0.4964
Test loss: 0.4976378381252289
Test accuracy: 0.8317999839782715


In [ ]:
model_path_test = os.path.join(os.getcwd(), 'saved_models', 'keras_cifar10_trained_model.h5')
if os.path.exists(model_path_test):
    print("✅ 模型已成功儲存於：", model_path_test)
else:
    print("❌ 模型尚未儲存，請檢查訓練是否完成或資料夾是否建立。")

✅ 模型已成功儲存於： /content/saved_models/keras_cifar10_trained_model.h5


In [ ]:
training_loss = model_history.history['loss']
val_loss = model_history.history['val_loss']

plt.plot(training_loss, label="training_loss")
plt.plot(val_loss, label="validation_loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("Learning Curve")
plt.legend(loc='best')
plt.show()

In [ ]:
training_loss = model_history.history['accuracy']
val_loss = model_history.history['val_accuracy']

plt.plot(training_loss, label='Training Accuracy')
plt.plot(val_loss, label='Validation Accuracy')
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend()
plt.grid(True)
plt.show()

 **Copy model we trained before to Google Drive.**

**Next time, you can use the model we trained before directly for prediction.**

In [ ]:
# 掛載使用者的Google雲端硬碟 (drive/MyDrive) 到 Colab 的工作磁碟
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import shutil

In [ ]:
print(os.listdir(Path.cwd()))

print(Path.cwd())

print(os.listdir(Path.cwd() / 'saved_models'))

print(os.listdir(Path.cwd() / 'drive'))

['.config', 'drive', 'saved_models', 'sample_data']
/content
['keras_cifar10_trained_model.h5']
['MyDrive', '.shortcut-targets-by-id', '.Trash-0', '.Encrypted']


In [ ]:
target_path = Path.cwd() / 'drive' / 'MyDrive' / 'keras_cifar10_trained_model_No_Stopping.h5'
source_path = Path.cwd() / 'saved_models' / model_name

shutil.move(source_path, target_path)

PosixPath('/content/drive/MyDrive/keras_cifar10_trained_model_No_Stopping.h5')